##### Register and Validate Model
 - Register Model and set tag to {"validation_status":"pending"}
 - Validate model, set alias to production and chnage tag to validated

In [71]:
import sys
import os
import logging

# Dynamically add the project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)

In [72]:
from validations.validate import RegisterEvaluateModel

##### Get Environmental Variables

In [ ]:
# Get enviornment variables
dbutils.widgets.text("env", "dev")
dbutils.widgets.text("model_name", "dev_occupancymodel")
dbutils.widgets.text("catalog_name", "devaiml")
dbutils.widgets.text("schema_name", "occupancy_project")
dbutils.widgets.text("experiment_name", "dev_ProjectOccupancy")
dbutils.widgets.text("validation_data_path", "dbfs:/FileStore/ValidationData/occupancy_mlops_validation_data")

env = dbutils.widgets.get("env")
model_name = dbutils.widgets.get("model_name")
catalog_name = dbutils.widgets.get("catalog_name")
schema_name = dbutils.widgets.get("schema_name")
experiment_name = dbutils.widgets.get("experiment_name")
validation_data_path = dbutils.widgets.get("validation_data_path")
# get best run_id that can be inferred from the job
best_run_id = dbutils.jobs.taskValues.get("Train-Model","best_run_id", debugValue="")

##### Register and Evaluate Model

In [ ]:
if __name__ == "__main__":
    # define model name with 3 name space
    registered_model_name = f"{catalog_name}.{schema_name}.{model_name}"

    # get username from databricks notebook context
    username = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().get("user").get()
    try:
        # start model model evaluation
        evaluate = RegisterEvaluateModel(
            registered_model_name,
            experiment_name,
            username,
            validation_data_path
        )
        # register the model
        model_info = evaluate.register_model(best_run_id)
        # evaluate and assign alias
        evaluate.evaluate_model(model_info)
        logging.info(f"Model Evaluation completed for: {registered_model_name}")
    except Exception as e:
        logging.error(f"Failed to complete {registered_model_name} Evaluation")
        raise RuntimeError(f"Error during model evaluation: {e}")